# GOAL
Identify non professional TalentCards users in GBQ.

# PACKAGES

In [42]:
import pandas as pd
from google.oauth2 import service_account
import pandas_gbq
import yaml

# PARAMETERS

In [2]:
project_id = "analytics-dev-308300"

credentials = service_account.Credentials.from_service_account_file(
    "../keys/gcp_key.json",
)

# FUNCTIONS

# DATA WRANGLING

CSV file was manually generated:
- extract list of non-pro users from TalentCards (filter)
- manipulate the excel file to include the assotiation and clean any data besides identifier and association
- export as csv

In [69]:
df_non_pro=pd.read_csv('../params/non_professional_users.csv',sep=';')
df_non_pro['Identifier']=df_non_pro['Identifier'].replace({'-':''},regex=True)
df_non_pro

,Identifier,Association
0,AQGJEGFMZPFF,Humane
1,AQZWKMAQHJFL,Danone
2,AQCAGKPUKWUI,Humane
3,AQAISHKLLSMM,Humane
4,AQNCCPYYWMAN,Danone
5,AQSNUTGKIPXA,Danone
6,AQOJUJAYOCKL,Danone
7,AQBJIIYALBBE,Humane
8,AQHQBUDACZMX,Humane
9,AQQOKIWJESYZ,Danone


In [70]:
query = """
SELECT * FROM dtm_engagement.dim_users
"""

df_bq_users=pd.read_gbq(query=query, credentials=credentials, project_id=project_id)

In [71]:
df_bq_users_flagged=df_bq_users.merge(df_non_pro,left_on='access_token',right_on='Identifier').drop(columns='Identifier').drop(columns=['access_token',	'email',	'first_name',	'last_name',	'mobile',	'created_at',	'last_login',	'updated_at',	'date_str',	'date_str_sp',	'extraction_date',	'last_login_date','group_id'])

In [72]:
df_bq_users_flagged

,user_id,Association
0,20029,Humane
1,20029,Humane
2,20030,Humane
3,20032,Humane
4,20100,Humane
5,20118,Humane
6,20129,Humane
7,20136,Danone
8,20144,Danone
9,20145,Danone


In [73]:
df_bq_users_flagged.to_gbq('talentcards.non_pro_users',if_exists='replace')

1it [00:04,  4.81s/it]
